In [22]:
from influxdb import InfluxDBClient

client = InfluxDBClient('localhost', 8086, 'root', '', 'testDB')
print(client.get_list_database())

[{'name': '_internal'}, {'name': 'testDB'}, {'name': 'NOAA_water_database'}, {'name': 'pyexample'}]


In [2]:
client.create_database("testDB")

In [3]:
result = client.query("show measurements;")
print("Result: {0}".format(result))

Result: ResultSet({'('measurements', None)': [{'name': 'weather'}]})


In [5]:
json_body = [
    {
        "measurement": "students",
        "tags": {
            "stuid": "s123"
        },
        #"time": "2017-03-12T22:00:00Z",
        "fields": {
            "score": 89
        }
    }
]
client.write_points(json_body) # 写入数据，同时创建表

True

In [6]:
result = client.query('select * from students;')    
print("Result: {0}".format(result))

Result: ResultSet({'('students', None)': [{'time': '2020-03-01T10:45:32.249327Z', 'score': 89, 'stuid': 's123'}]})


In [2]:
client.create_database('pyexample')

In [3]:
client.get_list_database()

[{'name': '_internal'},
 {'name': 'testDB'},
 {'name': 'NOAA_water_database'},
 {'name': 'pyexample'}]

In [4]:
client.switch_database('pyexample')

In [17]:
json_body = [
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-28T8:01:00Z",
        "fields": {
            "duration": 127
        }
    },
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-29T8:04:00Z",
        "fields": {
            "duration": 132
        }
    },
    {
        "measurement": "brushEvents",
        "tags": {
            "user": "Carol",
            "brushId": "6c89f539-71c6-490d-a28d-6c5d84c0ee2f"
        },
        "time": "2020-03-30T8:02:00Z",
        "fields": {
            "duration": 129
        }
    }
]
client.write_points(json_body)

True

In [18]:
results=client.query('SELECT "duration" FROM "pyexample"."autogen"."brushEvents" WHERE time > now() - 4d GROUP BY "user"')
results.raw

{'statement_id': 0,
 'series': [{'name': 'brushEvents',
   'tags': {'user': 'Carol'},
   'columns': ['time', 'duration'],
   'values': [['2020-03-01T19:14:14.552878Z', 129],
    ['2020-03-28T08:01:00Z', 127],
    ['2020-03-29T08:04:00Z', 132],
    ['2020-03-30T08:02:00Z', 129]]}]}

In [19]:
points = results.get_points(tags={'user':'Carol'})
for point in points:
    print("Time: %s, Duration: %i" % (point['time'], point['duration']))

Time: 2020-03-01T19:14:14.552878Z, Duration: 129
Time: 2020-03-28T08:01:00Z, Duration: 127
Time: 2020-03-29T08:04:00Z, Duration: 132
Time: 2020-03-30T08:02:00Z, Duration: 129


In [20]:
>>> results = client.query('SELECT "duration" FROM "pyexample"."autogen"."brushEvents" WHERE time > now() - 4d GROUP BY "brushId"')
>>> points = results.get_points(tags={'brushId': '6c89f539-71c6-490d-a28d-6c5d84c0ee2f'})
>>> brush_usage_total = 0
>>> for point in points:
...     brush_usage_total = brush_usage_total + point['duration']

In [21]:
>>> if brush_usage_total > 350:
...     print("You've used your brush head for %s seconds, more than the recommended amount! Time to replace your brush head!" % brush_usage_total)

You've used your brush head for 517 seconds, more than the recommended amount! Time to replace your brush head!
